In [1]:
! pip3 install -q --upgrade --user google-cloud-aiplatform==1.60.0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)

qwiklabs-gcp-00-af66386a640e


In [2]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [3]:
def add_num(a, b):
    print("Calling add function")
    return a+b
def mul_num(a, b):
    print("Calling multiply function")
    return a*b

In [4]:
# Function declaration for add_num
add_num_func = FunctionDeclaration(
    name="add_num",
    description="Add two numbers",
    parameters={
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"}
        },
    },
)

# Function declaration for mul_num
mul_num_func = FunctionDeclaration(
    name="mul_num",
    description="Multiply two numbers",
    parameters={
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"}
        },
    },
)


In [5]:
# Define a math Tool with add and multiply functions
math_tool = Tool(
    function_declarations=[add_num_func, mul_num_func],
)


In [36]:
# Initialize the model with gemini-1.5-pro-001
model = GenerativeModel(
    model_name="gemini-1.5-pro-001",
    tools=[math_tool],  # Include the math tool
    generation_config={"temperature": 0},  # Set temperature to 0
    system_instruction=[
        "Fulfill the user's instructions.",
        "If asked to add or multiply numbers, call the provided functions.",
        "You may call one function after the other if needed.",
        "Repeat the result to the user.",
        "If there is a scenario of summing up of things and then multiplying use both the functions provided to get the best results",
    ]
)


In [37]:
chat = model.start_chat()

In [38]:
def handle_response(response):
    # If there is a function call then invoke it
    if response.candidates[0].function_calls:
        function_call = response.candidates[0].function_calls[0]
    else:
        print(response.text)
        return
    
    # Check if the function_call requests your add function
    if function_call.name == "add_num":
        # Extract the arguments to use in your function
        a = function_call.args["a"]
        b = function_call.args["b"]
        # Call your function
        result = add_num(a, b)
        # Send the result back to the chat session with the model
        response = model.generate_content(
            Content(
                role="user",
                parts=[Part.from_text(f"The result of adding {a} and {b} is {result}.")],
            ),
        )
        # Make a recursive call of this handler function
        handle_response(response)

    # Check if the function_call requests your multiply function
    elif function_call.name == "mul_num":
        # Extract the arguments to use in your function
        a = function_call.args["a"]
        b = function_call.args["b"]
        # Call your function
        result = mul_num(a, b)
        # Send the result back to the chat session with the model
        response = model.generate_content(
            Content(
                role="user",
                parts=[Part.from_text(f"The result of multiplying {a} and {b} is {result}.")],
            ),
        )
        # Make a recursive call of this handler function
        handle_response(response)

    else:
        # You shouldn't end up here
        print(function_call)


In [39]:
response = chat.send_message("Tell me a joke.")
handle_response(response)

Why don't scientists trust atoms?

Because they make up everything! 



In [40]:
response = chat.send_message("I have 7 pizzas each with 16 slices. How many slices do I have?")
handle_response(response)

Calling multiply function
You are right, the product of 7.0 and 16.0 is indeed 112.0. 



In [41]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. How many pizzas did they bring together?")
handle_response(response)

Calling add function
You are right, the result of adding 3.0 and 4.0 is indeed 7.0. 



In [42]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. There are 16 slices per pizza. How many slices are there?")
handle_response(response)  

Calling add function
You are right, the result of adding 3.0 and 4.0 is 7.0. 



In [43]:
response = chat.send_message("Doug brought 4 pizzas, but Andrew dropped 2 on the ground. How many pizzas are left?")
handle_response(response) 

Calling add function
Calling add function
You are right, the result of adding 4.0 and -2.0 is 2.0. 

